In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf
from tensorflow.python.framework import ops
ops.disable_eager_execution()

import sys 
import yaml
import math
import time
import random
import traceback
import xarray as xr
import numpy as np
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt
import scipy.sparse
from scipy.ndimage import gaussian_filter

from tqdm.auto import tqdm

import numpy.fft as FFT
from typing import List, Dict

from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
from tensorflow.keras.layers import (Input, Conv2D, Dense, Flatten, BatchNormalization,
                                     MaxPool2D, RepeatVector, Lambda, Activation,
                                     LeakyReLU, Dropout)
from tensorflow.keras.models import Model, save_model
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow.keras.backend as K

from keras_radam import RAdam
from keras_radam.training import RAdamOptimizer

from holodecml.library.losses import SymmetricCrossEntropy
from holodecml.library.callbacks import get_callbacks
from holodecml.library.FourierOpticsLib import OpticsFFT, OpticsIFFT

from multiprocessing import cpu_count, Pool

Using TensorFlow backend.


### Load the configuration

In [2]:
config_file = "config.yml"

In [3]:
with open(config_file) as config_file:
    config = yaml.load(config_file, Loader=yaml.FullLoader)

In [4]:
try:
    os.makedirs(config["path_save"])
except Exception as E:
    pass

### Set up some globals

In [5]:
path_data = config["path_data"]
num_particles = config["num_particles"]
split = 'train'
subset = False
output_cols = ["x", "y", "z", "d", "hid"] 

batch_size = config["conv2d_network"]["batch_size"]

input_shape = (600, 400, 4)

n_particles = config["num_particles"]
output_channels = len(output_cols) - 1

maxnum_particles = 3

### Set up the logger

In [6]:
root = logging.getLogger()
root.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(levelname)s:%(name)s:%(message)s')

# Stream output to stdout
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
root.addHandler(ch)
#########

# Save the log file
fp = os.path.join('run2/log.txt')
fh = logging.FileHandler(fp,
                         mode='a+',
                         encoding='utf-8')
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
root.addHandler(fh)

In [7]:
num_particles_dict = {
    1: '1particle',
    3: '3particle',
    'multi': 'multiparticle',
    '50-100': '50-100'}

split_dict = {
    'train' : 'training',
    'test'   : 'test',
    'valid': 'validation'}


class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(
        
        self, 
        path_data: str, 
        num_particles: int, 
        split: str, 
        subset: bool, 
        output_cols: List[str], 
        batch_size: int, 
        shuffle: bool = True,
        maxnum_particles: int = False,
        scaler: Dict[str, str] = False) -> None:
        
        'Initialization'
        self.ds = self.open_dataset(path_data, num_particles, split)
        self.batch_size = batch_size
        self.output_cols = [x for x in output_cols if x != 'hid']        
        self.subset = subset
        self.hologram_numbers = self.ds.hologram_number.values
        if shuffle:
            random.shuffle(self.hologram_numbers)
        self.num_particles = num_particles
        self.xsize = len(self.ds.xsize.values)
        self.ysize = len(self.ds.ysize.values)
        self.shuffle = shuffle
        self.maxnum_particles = maxnum_particles
                
        if not scaler:
            self.scaler = {col: StandardScaler() for col in output_cols}
            for col in output_cols:
                scale = self.ds[col].values
#                 if col == "x":
#                     scale = np.array([(x + 888) / 2.96 for x in scale])
#                 if col == "y":
#                     scale = np.array([(y + 592) / 2.96 for y in scale])
                self.scaler[col].fit(scale.reshape(scale.shape[-1], -1))
        else:
            self.scaler = scaler
        
    def get_transform(self):
        return self.scaler

    def __len__(self):
        'Denotes the number of batches per epoch'
        return math.ceil(len(self.hologram_numbers) / self.batch_size)
    
    def __getitem__(self, idx):
        'Generate one batch of data'
        holograms = self.hologram_numbers[
            idx * self.batch_size: (idx + 1) * self.batch_size
        ]
        x_out, y_out, w_out = self._batch(holograms)
        return x_out, y_out
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle == True:
            random.shuffle(self.hologram_numbers)
            
    def standardize(self, X):
        X = (X-np.mean(X)) / (np.std(X))
        return X
    
    def reshape(self, X):
        x, y = X.shape
        return X.reshape((x,y,1))
            
    def _batch(self, holograms: List[int]):
        'Create a batch of data'
        try:
            x_out = np.zeros((
                len(holograms), self.xsize, self.ysize, 4
            ))
            y_out = np.zeros((
                len(holograms), 
                self.maxnum_particles if self.maxnum_particles else self.num_particles, 
                len(self.output_cols)
            ))
            b_out = np.zeros((
                len(holograms), 
                self.maxnum_particles if self.maxnum_particles else self.num_particles, 
                1
            ))
            a = time.time()
            for k, hologram in enumerate(holograms):
                #im = self.ds["image"][hologram].values
                #x_out[k] = (im-np.mean(im)) / (np.std(im))
                
                im = self.ds["image"][hologram].values
                A = self.standardize(im)
                F = OpticsFFT(A)                     
                R = self.reshape(self.standardize(np.log(np.abs(F))))
                P = self.reshape(self.standardize(np.real(F)))
                Q = self.reshape(self.standardize(np.imag(F)))
                x_out[k] = np.concatenate((self.reshape(A), R, P, Q), axis = 2)
                
                particles = np.where(self.ds["hid"] == hologram + 1)[0]  
                for l, p in enumerate(particles):
                    for m, col in enumerate(self.output_cols):
                        val = self.ds[col].values[p]
#                         if col == "x":
#                             val = (val + 888) / 2.96
#                         if col == "y":
#                             val = (val + 592) / 2.96
                        
                        y_out[k, l, m] = self.scaler[col].transform(
                            val.reshape(1, -1)
                        )
                    b_out[k, l, 0] = 1
                    
                if self.maxnum_particles and len(particles) < self.maxnum_particles:
                    for l in range(len(particles), self.maxnum_particles):
                        for m, col in enumerate(self.output_cols):
                            val = y_out[k, l, m]
                            y_out[k, l, m] = self.scaler[col].transform(
                                val.reshape(1, -1)
                            )
                            
                    #b_out[k, l, 0] = 0
            #x_out = np.expand_dims(x_out, axis=-1)
            return x_out, [b_out, y_out], [[None], [None]] #class weights option
        
        except:
            print(traceback.print_exc())
    
    def open_dataset(self, path_data, num_particles, split):
        """
        Opens a HOLODEC file

        Args: 
            path_data: (str) Path to dataset directory
            num_particles: (int or str) Number of particles per hologram
            split: (str) Dataset split of either 'train', 'valid', or 'test'

        Returns:
            ds: (xarray Dataset) Opened dataset
        """
        path_data = os.path.join(path_data, self.dataset_name(num_particles, split))

        if not os.path.isfile(path_data):
            print(f"Data file does not exist at {path_data}. Exiting.")
            raise 

        ds = xr.open_dataset(path_data)
        return ds
    
    def dataset_name(self, num_particles, split, file_extension='nc'):
        """
        Return the dataset filename given user inputs

        Args: 
            num_particles: (int or str) Number of particles per hologram
            split: (str) Dataset split of either 'train', 'valid', or 'test'
            file_extension: (str) Dataset file extension

        Returns:
            ds_name: (str) Dataset name
        """

        valid = [1,3,'multi','50-100']
        if num_particles not in valid:
            raise ValueError("results: num_particles must be one of %r." % valid)
        num_particles = num_particles_dict[num_particles]

        valid = ['train','test','valid']
        if split not in valid:
            raise ValueError("results: split must be one of %r." % valid)
        split = split_dict[split]
        
        if num_particles == "50-100":
            ds_name = f'synthetic_holograms_{num_particles}particle_monodisperse_{split}.{file_extension}'
        else:
            ds_name = f'synthetic_holograms_{num_particles}_{split}.{file_extension}'
        return ds_name

In [8]:
train_gen = DataGenerator(
    path_data, num_particles, "train", subset, 
    output_cols, batch_size, maxnum_particles = maxnum_particles, 
    shuffle = True
)

In [9]:
# scaler = {col: StandardScaler() for col in train_gen.output_cols}
# for col in train_gen.output_cols:
#     scale = train_gen.ds[col].values
#     scaler[col].fit(scale.reshape(scale.shape[-1], -1))
#     result = scaler[col].transform(scale.reshape(scale.shape[-1], -1))
#     print(col, min(result), max(result), np.mean(result), np.std(result))

In [10]:
train_scalers = train_gen.get_transform()

In [11]:
valid_gen = DataGenerator(
    path_data, num_particles, "test", subset, 
    output_cols, batch_size, scaler = train_scalers, 
    maxnum_particles = maxnum_particles, shuffle = False
)

### Initialize callbacks

In [12]:
callbacks = get_callbacks(config)

### Set up a model

##### Custom losses

In [13]:
#@tf.function
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

#@tf.function
def R2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return SS_res/(SS_tot + K.epsilon())

#@tf.function
def keras_mse(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))

#@tf.function
def wsme(y_true, y_pred):
    #w = K.abs(K.mean(y_true[1]))
    #w = w / (1 - w)
    # w = K.sum(K.cast(K.greater(y_true[1], 0), "float")) # Number actually not zero
    #error = K.square(y_true - y_pred)
    #error = K.switch(K.equal(y_true, 0), w * error, error)
    y_true = y_true * (y_true != 0) 
    y_pred = y_pred * (y_true != 0)
    return keras_mse(y_true, y_pred)

In [14]:
n_particles = maxnum_particles

custom_losses = {
    "sce": SymmetricCrossEntropy(0.5, 0.5),
    "weighted_mse": wsme,
    "r2": R2,
    "rmse": rmse
}

class Conv2DNeuralNetwork(object):
    """
    A Conv2D Neural Network Model that can support an arbitrary numbers of
    layers.

    Attributes:
        filters: List of number of filters in each Conv2D layer
        kernel_sizes: List of kernel sizes in each Conv2D layer
        conv2d_activation: Type of activation function for conv2d layers
        pool_sizes: List of Max Pool sizes
        dense_sizes: Sizes of dense layers
        dense_activation: Type of activation function for dense layers
        output_activation: Type of activation function for output layer
        lr: Optimizer learning rate
        optimizer: Name of optimizer or optimizer object.
        adam_beta_1: Exponential decay rate for the first moment estimates
        adam_beta_2: Exponential decay rate for the first moment estimates
        sgd_momentum: Stochastic Gradient Descent momentum
        decay: Optimizer decay
        loss: Name of loss function or loss object
        batch_size: Number of examples per batch
        epochs: Number of epochs to train
        verbose: Level of detail to provide during training
        model: Keras Model object
    """
    def __init__(
        self, 
        filters=(8,), 
        kernel_sizes=(5,),
        conv2d_activation="relu", 
        pool_sizes=(4,), 
        pool_dropout=0.0,
        dense_sizes=(64,),
        dense_activation="relu", 
        dense_dropout = 0.0,
        output_activation="linear",
        lr=0.001, 
        optimizer="adam", 
        adam_beta_1=0.9,
        adam_beta_2=0.999, 
        sgd_momentum=0.9, 
        decay=0, 
        loss="mse",
        metrics = [], 
        batch_size=32, 
        epochs=2, 
        verbose=0
    ):
        
        self.filters = filters
        self.kernel_sizes = [tuple((v,v)) for v in kernel_sizes]
        self.conv2d_activation = conv2d_activation
        self.pool_sizes = [tuple((v,v)) for v in pool_sizes]
        self.pool_dropout = pool_dropout
        self.dense_sizes = dense_sizes
        self.dense_activation = dense_activation
        self.dense_dropout = dense_dropout
        self.output_activation = output_activation
        self.lr = lr
        self.optimizer = optimizer
        self.optimizer_obj = None
        self.adam_beta_1 = adam_beta_1
        self.adam_beta_2 = adam_beta_2
        self.sgd_momentum = sgd_momentum
        self.decay = decay
        self.loss = custom_losses[loss] if loss in custom_losses else loss
        self.metrics = []
        for m in metrics:
            if m in custom_losses:
                self.metrics.append(custom_losses[m])
            else:
                self.metrics.append(m)
        self.batch_size = batch_size
        self.epochs = epochs
        self.verbose = verbose
        self.model = None
        
        if self.conv2d_activation == "leakyrelu":
            self.conv2d_activation = LeakyReLU(alpha=0.1)
        if self.dense_activation == "leakyrelu":
            self.dense_activation = LeakyReLU(alpha=0.1)
        if self.output_activation == "leakyrelu":
            self.output_activation = LeakyReLU(alpha=0.1)

    def build_neural_network(self, input_shape, n_particles, output_shape):
        """Create Keras neural network model and compile it."""
        
        # function for creating a vgg block
        def vgg_block(layer_in, n_filters, n_conv):
            # add convolutional layers
            for _ in range(n_conv):
                layer_in = Conv2D(n_filters, (3, 3), 
                                  padding='same', activation='linear')(layer_in)
                layer_in = Activation('relu')(layer_in)
            
            # Batch norm
            layer_in = BatchNormalization(axis=-1)(layer_in)
            # Dropout
            #layer_in = Dropout(0.2)(layer_in)
            # Max pooling 
            layer_in = MaxPool2D((2, 2), strides=(2, 2))(layer_in)
            return layer_in
        
        # Input
        conv_input = Input(shape=(input_shape), name="input")
        
        nn_model = vgg_block(conv_input, 32, 2)
        nn_model = vgg_block(nn_model, 64, 2)
        nn_model = vgg_block(nn_model, 128, 4)
        #nn_model = vgg_block(nn_model, 256, 4)        
        nn_model = Flatten()(nn_model)
        
#         nn_model = BatchNormalization(axis=-1)(nn_model)
#         nn_model = Dropout(0.5)(nn_model)
        
#         # Classifier
#         for h in range(len(self.dense_sizes)):
#             nn_model = Dense(self.dense_sizes[h],
#                              activation=self.dense_activation,
#                              kernel_initializer='he_uniform',
#                              name=f"dense_{h:02d}")(nn_model)
#             if self.dense_dropout > 0.0:
#                 nn_model = Dropout(self.dense_dropout, 
#                                    name=f"dense_dr_{h:02d}")(nn_model)
        
        nn_model = RepeatVector(n_particles, name = "repeat")(nn_model)
                
        # Output 1 - particle prediction
        binary_model = Dense(1, 
                             activation="sigmoid", 
                             kernel_initializer='he_uniform',
                             name=f"binary")(nn_model)
        
        # Output 2 - configuration prediction
        nn_model = Dense(output_shape,
                         activation="linear",
                         kernel_initializer='he_uniform',
                         name=f"coordinate")(nn_model)
        
#         nn_model = Lambda(
#             self.LastLayer,
#             input_shape = (n_particles, output_shape),
#             name="coordinate"
#         )(nn_model)
        
        self.model = Model(
            inputs = conv_input, 
            outputs = [binary_model, nn_model]
        )
        
        losses = {
            "binary": "binary_crossentropy",
            "coordinate": "mse" #custom_losses["weighted_mse"]
        }
        lossWeights = {"binary": 1.0, "coordinate": 1.0}
        
        if self.optimizer == "adam":
            self.optimizer_obj = Adam(lr=self.lr, clipnorm = 5.0)
        elif self.optimizer == "sgd":
            self.optimizer_obj = SGD(lr=self.lr, momentum=self.sgd_momentum,
                                     decay=self.decay)
            
        self.model.compile(
            optimizer=self.optimizer_obj, 
            loss=losses, #self.loss,
            loss_weights=lossWeights,
            metrics=self.metrics
        )
        #self.model.summary()

    def fit(self, x, y, xv=None, yv=None, callbacks=None):
        
        if len(x.shape[1:])==2:
            x = np.expand_dims(x, axis=-1)
        if len(y.shape) == 1:
            output_shape = 1
        else:
            output_shape = y.shape[1]
        
        input_shape = x.shape[1:]
        self.build_neural_network(input_shape, output_shape)
        self.model.fit(x, y, batch_size=self.batch_size, epochs=self.epochs,
                       verbose=self.verbose, validation_data=(xv, yv), callbacks=callbacks)
        return self.model.history.history
    
    def LastLayer(self, x):
        return 1.75 * K.tanh(x / 100) 

    def predict(self, x):
        y_out = self.model.predict(np.expand_dims(x, axis=-1),
                                   batch_size=self.batch_size)
        return y_out

    def predict_proba(self, x):
        y_prob = self.model.predict(x, batch_size=self.batch_size)
        return y_prob
    
    def load_weights(self, weights):
        try:
            self.model.load_weights(weights)
            self.model.compile(
                optimizer=self.optimizer, 
                loss=self.loss, 
                metrics=self.metrics
            )
        except:
            print("You must first call build_neural_network before loading weights. Exiting.")
            sys.exit(1)

In [15]:
mod = Conv2DNeuralNetwork(**config["conv2d_network"])

In [16]:
mod.build_neural_network(input_shape, n_particles, output_channels)

In [17]:
mod.model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 600, 400, 4) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 600, 400, 32) 1184        input[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, 600, 400, 32) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 600, 400, 32) 9248        activation[0][0]                 
______________________________________________________________________________________________

### Train a model

In [18]:
mod.model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    epochs=config["conv2d_network"]["epochs"],
    verbose=True,
    callbacks=callbacks,
    #steps_per_epoch=20,
    use_multiprocessing=True,
    workers=16,
    max_queue_size=32
)

Epoch 1/1000


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[128,32,600,400] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node activation/Relu}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Func/training/Adam/gradients/gradients/batch_normalization_1/cond_grad/StatelessIf/else/_57/input/_231/_345]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[128,32,600,400] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node activation/Relu}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current a